YÖK Tez Veritabanı'ndan tez bilgisi alma
POST şekli: neden=mehmet+selim+yavuz&nevi=2&izin=0&tur=0&islem=1&kaydet=+++++Bul++++++ #nevi = cinsi 1 tez adı, 2 yazar

In [8]:
import requests
import re
from bs4 import BeautifulSoup
url = 'https://tez.yok.gov.tr/UlusalTezMerkezi/'
page = requests.get('https://tez.yok.gov.tr/UlusalTezMerkezi/').text
soup = BeautifulSoup(page, 'html.parser')
forms = soup.find_all('form')
for form in forms:
    #print(form['action'])
    url = url + form['action']
    #print(url)
def tezleriAl (akademisyen):
    payload = {'neden': akademisyen, 'nevi':'2', 'izin': '0', 'tur': '0', 'islem': '1', 'kaydet':'+++++Bul++++++'} #buraya like search ekle

    def post_request(url, payload):
        try:
            r = requests.post(url, data=payload)
            if r.status_code == 200:
                return r
        except Exception as e:
            print(e)

    tezResponsePage = post_request(url, payload).text
    tezSoup = BeautifulSoup(tezResponsePage, 'html.parser')
    tezListScript = tezSoup.find_all('script', type='text/javascript')
    cleanScript = str(tezListScript[-1]).replace('\n', '')
    cleanScript = cleanScript.replace('\t', '')
    cleanScript = cleanScript.replace('  ', '')
    cleanScript = cleanScript.replace('{', '***')
    cleanScript = cleanScript.replace('}', '***')
    splitScript = cleanScript.split('***')
    length = len(splitScript)
    #len(splitScript)

    i = 0
    toBeSelected = []
    while i < length:
        if splitScript[i].startswith('rowNum:') == True:
            if splitScript[i].startswith('rowNum: ') != True:
                toBeSelected.append(i)
        i = i + 1
    cleanTezList = []
    for i in toBeSelected:
        cleanTezList.append(splitScript[i])
    #print(cleanTezList)

    itemizedList = []
    for item in cleanTezList:
        splitIt = item.split(',')
        splitIt = splitIt[:-1]
        itemizedList.append(splitIt)
    #print(itemizedList)

    tezDict = []
    for tez in itemizedList:
        #print(tez)
        itemDict = {}
        for item in tez:
            #print(item)
            splitted = item.split(':')
            if len(splitted) == 1:
                itemDict['no'] = splitted[0]
            elif len(splitted) == 2:
                if splitted[1][0] == ' ':
                    if splitted[1][1] == '\"':
                        itemDict[splitted[0]] = splitted[1][2:-1]
                    else:
                        itemDict[splitted[0]] = splitted[1][1:]
                else:
                    if splitted[1][0] == '\"':
                        itemDict[splitted[0]] = splitted[1][1:-1]
                    else:
                        itemDict[splitted[0]] = splitted[1]
            else:
                unsplitted = ''
                for part in splitted[1:]:
                    unsplitted = unsplitted + part
                if unsplitted[1] == '\"':
                    itemDict[splitted[0]] = unsplitted[2:-1]
                else:
                    itemDict[splitted[0]] = unsplitted
                #print(splitted)
        tezDict.append(itemDict)
    for index,tez in enumerate(tezDict):
        tezDict[index]['tezNo'] = tezDict[index].pop('rowNum')
        tezDict[index]['id'] = tezDict[index].pop(' userId')
        tezDict[index]['no'] = tezDict[index].pop('no')
        tezDict[index]['isim'] = tezDict[index].pop(' name')
        tezDict[index]['tezYili'] = tezDict[index].pop('age')
        tezDict[index]['tezBasligi'] = tezDict[index].pop(' weight')
        tezDict[index]['uni'] = tezDict[index].pop(' uni')
        tezDict[index]['tezDili'] = tezDict[index].pop(' height')
        tezDict[index]['tezCinsi'] = tezDict[index].pop(' important')
        tezDict[index]['tezKonusu'] = tezDict[index].pop(' someDate')
    for index, tez in enumerate(tezDict):
        tezDict[index]['id'] = tezDict[index]['id'].split('tezDetay(\'')[-1]
        tezDict[index]['tezBasligi'] = tezDict[index]['tezBasligi'].split('<br>')[0]
        yeniNo = tezDict[index]['no'].split("\\'")[0]
        yeniNo = yeniNo.split("'")[1]
        tezDict[index]['no'] = yeniNo
        if len(tezDict[index]['tezKonusu'].split(' = ')) != 2:
            konu = tezDict[index]['tezKonusu'].split(' ; ')
            konuList = []
            for alan in konu:
                coklu = alan.split(' = ')
                konuList.append(coklu[0])
            tezDict[index]['tezKonusu'] = konuList
        else:
            konu = tezDict[index]['tezKonusu'].split(' = ')
            tezDict[index]['tezKonusu'] = [konu[0]]
    for index, tez in enumerate(tezDict):
        if 'Müzik' not in tezDict[index]['tezKonusu']:
            tezDict.pop(index)
    return(tezDict)

In [9]:
currentAkademisyen = tezleriAl('seyit yöre')

In [21]:
url = 'https://tez.yok.gov.tr/UlusalTezMerkezi/tezDetay.jsp'
for id, tez in enumerate(currentAkademisyen):
    payload = {'id': currentAkademisyen[id]['id'], 'no':currentAkademisyen[id]['no']}

    def post_request(url, payload):
        try:
            r = requests.post(url, data=payload)
            if r.status_code == 200:
                return r
        except Exception as e:
            print(e)

    detayResponsePage = post_request(url, payload).text
    detaySoup = BeautifulSoup(detayResponsePage, 'html.parser')
    for item in detaySoup.find_all('td'):
        if str(item).find(currentAkademisyen[id]['isim']) > 0:
            print('found the right table division for thesis #' + str(id))


found the right table division for thesis #0
found the right table division for thesis #1
